# Le notebook que yohan n'a pas fait

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import os

In [3]:
def merge_vehicle_count(caracteristiques, vehicules):
    # Compter le nombre de véhicules par "Num_Acc" dans le DataFrame vehicules
    vehicules_count = vehicules.groupby("Accident_Id")["id_vehicule"].count().reset_index().rename(columns={'id_vehicule':'Nombre_vehicule'})
    
    # Fusionner le DataFrame caracteristiques avec le DataFrame vehicules_count en utilisant la colonne "Num_Acc"
    merged_df = caracteristiques.merge(vehicules_count, on='Accident_Id')
    
    return merged_df


def merge_user_count(caracteristiques,usagers):
    # Compter le nombre d'usagers par "Accident_Id" et par sexe dans le DataFrame usagers
    usagers_count = usagers.groupby(["Accident_Id", "sexe"])["id_vehicule"].count().unstack(fill_value=0)
    usagers_count.rename(columns={1: 'Nombre_Hommes', 2: 'Nombre_Femmes'}, inplace=True)
    
    # Fusionner le DataFrame caracteristiques avec usagers_count en utilisant la colonne "Num_Acc"
    merged_df = caracteristiques.merge(usagers_count, on='Accident_Id')
    
    return merged_df

def replace_with_mapper(df, col, mapper):
    df[col] = df[col].replace(mapper)
    
    return df

# Remplacement des heures par le moment de la journée
matinee = ['06:','07:','08:','09:','10:']
journee = ['11:','12:','13:','14:','15:','16:','17:']
soiree = ['18:','19:','20:','21:','22:','23:']
nuit = ['00:','01:','02:','03:','04:','05:']

def replace_hour(hour):
    hour = hour[:3]
    if hour in matinee:
        return 'Matinée'
    elif hour in journee:
        return 'Journée'
    elif hour in soiree:
        return 'Soirée'
    elif hour in nuit:
        return 'Nuit'
    else:
        'Lucas'

In [4]:
years = ['19', '20', '21', '22']
files_name = ["caracteristiques-20", "lieux-20", "usagers-20", "vehicules-20"]
if not os.path.exists('./Dataset/all_years'):
    os.mkdir('./Dataset/all_years/', 0o666) 
for file_name in files_name:
    result = []
    print(f"\n///////////////{file_name}///////////////")
    for year in years:
        file_path = f'./Dataset/{year}/{file_name}{year}.csv'
        sep = ';'
        df = pd.read_csv(file_path, sep=sep, low_memory=False)
        # if 'id_usager' in df.columns:
        #     df = df.drop(columns=['id_usager'])
        result.append(df)
        print(df.columns.tolist())
    combined_df = pd.concat(result, ignore_index=True)
    combined_df.to_csv(f'./Dataset/all_years/{file_name}{years[0]}-{years[-1]}.csv', index=False)


///////////////caracteristiques-20///////////////
['Accident_Id', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long']
['Accident_Id', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long']
['Accident_Id', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long']
['Accident_Id', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long']

///////////////lieux-20///////////////
['Accident_Id', 'catr', 'voie', 'v1', 'v2', 'circ', 'nbv', 'vosp', 'prof', 'pr', 'pr1', 'plan', 'lartpc', 'larrout', 'surf', 'infra', 'situ', 'vma']
['Accident_Id', 'catr', 'voie', 'v1', 'v2', 'circ', 'nbv', 'vosp', 'prof', 'pr', 'pr1', 'plan', 'lartpc', 'larrout', 'surf', 'infra', 'situ', 'vma']
['Accident_Id', 'catr', 'voie', 'v1', 'v2', 'circ', 'nbv', 'vosp', 'prof', 'pr', 'pr1', 'plan', 'lartpc', 'larrout', 'surf', 'infra', 'situ', '

In [27]:
caracteristiques = pd.read_csv('./Dataset/all_years/caracteristiques-2019-22.csv')

caracteristiques = caracteristiques.drop(['com', 'adr', 'lat', 'long'], axis=1)

# Remplacer les numéros des mois par leurs noms
caracteristiques.mois = caracteristiques.mois.map({
    1:'Janvier',
    2:'Février',
    3:'Mars',
    4:'Avril',
    5:'Mai',
    6:'Juin',
    7:'Juillet',
    8:'Août',
    9:'Septembre',
    10:'Octobre',
    11:'Novembre',
    12:'Décembre'
}, na_action=None)



caracteristiques = replace_with_mapper(caracteristiques, 'lum', {
    1 : 'Plein jour',
    2 : 'Crépuscule ou aube',
    3 : 'Nuit sans éclairage public',
    4 : 'Nuit avec éclairage public non allumé',
    5 : 'Nuit avec éclairage public allumé'
})
caracteristiques = replace_with_mapper(caracteristiques, 'agg', {
    1 : 'Hors agglomération',
    2 : 'En agglomération'
})
caracteristiques = replace_with_mapper(caracteristiques, 'int', {
    1 : 'Hors intersection',
    2 : 'Intersection en X',
    3 : 'Intersection en T',
    4 : 'Intersection en Y',
    5 : 'Intersection à plus de 4 branches',
    6 : 'Giratoire',
    7 : 'Place',
    8 : 'Passage à niveau',
    9 : 'Autre intersection'
})
caracteristiques = replace_with_mapper(caracteristiques, 'atm', {
    -1 :'Non renseigné',
    1 : 'Normale',
    2 : 'Pluie légère',
    3 : 'Pluie forte',
    4 : 'Neige - grêle',
    5 : 'Brouillard - fumée',
    6 : 'Vent fort - tempête',
    7 : 'Temps éblouissant',
    8 : 'Temps couvert',
    9 : 'Autre'
})
caracteristiques = replace_with_mapper(caracteristiques, 'col', {
    -1 :'Non renseigné',
    1 : 'Deux véhicules - frontale',
    2 : 'Deux véhicules – par l’arrière',
    3 : 'Deux véhicules – par le coté',
    4 : 'Trois véhicules et plus – en chaîne',
    5 : 'Trois véhicules et plus - collisions multiples',
    6 : 'Autre collision',
    7 : 'Sans collision'
})

lieux = pd.read_csv('./Dataset/all_years/lieux-2019-22.csv')

caracteristiques = caracteristiques.merge(
replace_with_mapper(lieux.loc[:, ['Accident_Id', 'catr']], 'catr', {
    1:'Autoroute',
    2:'Route nationale',
    3:'Route Départementale',
    4:'Voie Communales',
    5:'Hors réseau public',
    6:'Parc de stationnementouvert à la circulation publique',
    7:'Routes de métropole urbaine',
    9:'autre'
}), on='Accident_Id', how='left')

caracteristiques = caracteristiques.merge(
replace_with_mapper(lieux.loc[:, ['Accident_Id', 'circ']], 'circ', {
    -1:'Non renseigné',
    1:'A sens unique',
    2:'Bidirectionnelle',
    3:'A chaussées séparées',
    4:'Avec voies d’affectation variable'
}), on='Accident_Id', how='left')

caracteristiques = caracteristiques.merge(lieux.loc[:, ['Accident_Id', 'nbv']], on='Accident_Id', how='left')

caracteristiques = caracteristiques.merge(
replace_with_mapper(lieux.loc[:, ['Accident_Id', 'surf']], 'surf', {
    -1:'Non renseigné',
    1:'Normale',
    2:'Mouillée',
    3:'Flaques',
    4:'Inondée',
    5:'Enneigée',
    6:'Boue',
    7:'Verglacée',
    8:'Corps gras – huile',
    9:'Autre'
}), on='Accident_Id', how='left')

caracteristiques = caracteristiques.merge(
replace_with_mapper(lieux.loc[:, ['Accident_Id', 'infra']], 'infra', {
    -1:'Non renseigné',
    0:'Aucun',
    1:'Souterrain -tunnel',
    2:'Pont -autopont',
    3:'Bretelle d’échangeur ou de raccordement',
    4:'Voie ferrée',
    5:'Carrefour aménagé',
    6:'Zone piétonne',
    7:'Zone de péage',
    8:'Chantier',
    9:'Autres'
}), on='Accident_Id', how='left')

caracteristiques = caracteristiques.merge(
replace_with_mapper(lieux.loc[:, ['Accident_Id', 'situ']], 'situ', {
    -1:'Non renseigné',
    0:'Aucun',
    1:'Sur chaussée',
    2:'Sur bande d’arrêt d’urgence',
    3:'Sur accotement',
    4:'Sur trottoir',
    5:'Sur piste cyclable',
    6:'Sur autre voie spéciale',
    8:'Autres'
}), on='Accident_Id', how='left')

caracteristiques = caracteristiques.merge(lieux.loc[:, ['Accident_Id', 'vma']], on='Accident_Id', how='left')

caracteristiques

,Accident_Id,jour,mois,an,hrmn,lum,dep,agg,int,atm,col,catr,circ,nbv,surf,infra,situ,vma
0,201900000001,30,Novembre,2019,01:30,Nuit avec éclairage public non allumé,93,Hors agglomération,Hors intersection,Normale,Deux véhicules – par l’arrière,Autoroute,A chaussées séparées,10,Normale,Pont -autopont,Sur chaussée,70
1,201900000002,30,Novembre,2019,02:50,Nuit sans éclairage public,93,Hors agglomération,Hors intersection,Normale,Autre collision,Autoroute,A sens unique,2,Normale,Aucun,Sur chaussée,70
2,201900000003,28,Novembre,2019,15:15,Plein jour,92,Hors agglomération,Hors intersection,Normale,Trois véhicules et plus – en chaîne,Autoroute,A chaussées séparées,8,Normale,Aucun,Sur chaussée,90
3,201900000004,30,Novembre,2019,20:20,Nuit avec éclairage public allumé,94,Hors agglomération,Hors intersection,Normale,Trois véhicules et plus – en chaîne,Autoroute,A chaussées séparées,5,Normale,Aucun,Sur chaussée,90
4,201900000005,30,Novembre,2019,04:00,Nuit sans éclairage public,94,Hors agglomération,Hors intersection,Normale,Deux véhicules – par l’arrière,Autoroute,A sens unique,3,Normale,Pont -autopont,Sur chaussée,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218399,202200055298,1,Janvier,2022,03:50,Nuit sans éclairage public,2B,Hors agglomération,Hors intersection,Normale,Autre collision,Route Départementale,Bidirectionnelle,2,Normale,Aucun,Autres,80
218400,202200055299,1,Janvier,2022,07:20,Nuit sans éclairage public,84,Hors agglomération,Hors intersection,Normale,Autre collision,Route Départementale,Bidirectionnelle,2,Normale,Aucun,Sur accotement,80
218401,202200055300,1,Janvier,2022,04:27,Nuit sans éclairage public,74,Hors agglomération,Hors intersection,Autre,Autre collision,Route Départementale,Bidirectionnelle,2,Verglacée,Aucun,Sur accotement,80
218402,202200055301,1,Janvier,2022,08:40,Plein jour,81,Hors agglomération,Intersection en T,Normale,Deux véhicules – par le coté,Route Départementale,Bidirectionnelle,2,Normale,Aucun,Sur chaussée,80
